In [113]:
from scipy import stats
import numpy as np
from scipy.stats import t
import matplotlib.pyplot as plt
import os
from os import path
import pandas as pd
import NMUtility

####################### start function defns ###########################
def get_table(path):
    table = pd.read_table(path,sep=",",skiprows=25, header=None,names=[0,1,2,3,4])
    table = table.drop([1,3,4], axis=1)
    return table

def collect_t_vals(table, num_models=2):
    #collect every item containing 'vs'
    compare_string = ''
    s = []
    t_on = 0
    d = {}
    for k,row in table.iterrows():
        col0 = row.iloc[0]
        col1 = row.iloc[1]
        
        try:
            if 'vs' in col0:
                compare_string = col0.replace("=","").strip()
                d[compare_string] = []
                s.append(compare_string)
                #print compare_string
        except Exception as e:
            pass

        #if col 1 of either of the two previous rows was 't', collect the value in col 1
        if t_on > num_models:
            t_on = 0
        elif t_on <= num_models and t_on > 0:
            d[compare_string].append(float(col1))
            t_on += 1

        if col1 == 't':
            # turn on flag to get the next two rows
            t_on += 1

    return s, d

def get_table_2(path):
    table = pd.read_table(path,sep=",",skiprows=25, header=None,names=[0,1,2,3,4])
    table = table.drop([1,3,4], axis=1)
    return table

def collect_t_vals_2(table, num_models):
    t_on = 0
    d={}
    for k, row in table.iterrows():
        if t_on > 0:
            key = str(row.iloc[0]).replace("=","").strip()
            d[key] =  float(row.iloc[1:].values[0])
        if row.iloc[1] == 't':
            # turn on flag to get the next two rows
            t_on += 1
    return d

def calc_real_ps(t_vals, df, mode=2):
    '''
    t_vals can be a number or an array-like. Equivalent to Open Offices TDIST function.
    #http://stackoverflow.com/questions/23879049/finding-two-tailed-p-value-from-t-distribution-
    and-degrees-of-freedom-in-python
    #http://stackoverflow.com/questions/17559897/python-p-value-from-t-statistic
    '''
    return mode*t.sf(t_vals, df)


def real_ps_1(measure_type):
    path = os.path.join(prism_out_dir, "2way ANOVA of {}.csv".format(measure_type))
    table = get_table(path)
    
    s,d = collect_t_vals(table)
    df = len(d)-1
    data = [d[i] for i in s]
    t_vals = pd.DataFrame(data, index=s, columns=['TB','Yan'])
    real_p_by_model = {}
    for k,v in t_vals.iteritems():
        x = v.values
        real_p_by_model[k+" p"] = calc_real_ps(x,df,2)
        #print k, real_p_by_model[k]
    return pd.concat([t_vals,pd.DataFrame(real_p_by_model, t_vals.index)], axis=1)
    return t_vals, real_p_by_model

def real_ps_2(measure_type):
    '''
    extract data from prism results output that compares models to each other
    '''
    #process file specified by path and create table
    path = os.path.join(prism_out_dir, "2way ANOVA of {} inv.csv".format(measure_type))
    table = get_table_2(path)
    
    header = table.iloc[0][0]
    t_values_dict = collect_t_vals_2(table,2)
    df = len(t_values_dict)-1
    t_values_series = pd.Series(t_values_dict.values(), t_values_dict.keys(), name=header)
    real_p = calc_real_ps(t_values_series.values,df,2)
    
    keys=[header+' t',header+' p']
    return pd.concat([t_values_series, pd.Series(real_p, t_values_series.index)], keys=keys,axis=1)


def convert_booleans(x):
    '''
    replace True with '*' and False with 'ns'
    '''
    
    x_arr = np.array(x)
    new_arr = []
    for n,i in enumerate(x):
        if i:
            new_arr.append('*')
        else:
            new_arr.append('ns')
    return new_arr

def compare_p_against_newalpha(frames, convert_bools_list=[]):
    '''
    This really needs to get broken up into smaller functions, probably
    convert_bools_list is a two item list. The first element will be used to replace True in the new column,
    the second will be used to replace False.
    '''
    
    #get the new alpha value
    num_comparisons = sum([len(frame) for frame in frames])
    new_alpha = 0.05/num_comparisons
    
    #compare each real p value to the new alpha
    #make a new column for the results of the comparison
    #append it to the dataframe
    compared_frames = []
    for frame in frames:
        
        #get the column names for the real p values
        #use boolean logic to get the new columns (by comparing them to the new_alpha)
        p_cols = [col_names for col_names in frame if ' p' in col_names]    
        p_lessthan_alpha = frame.loc[:,p_cols]<new_alpha

        #convert True and False according to scheme specified by convert_bools_list
        if convert_bools_list:
            p_lessthan_alpha.replace(to_replace=[True,False], value=convert_bools_list,inplace=True)
            
        # reset column names for new columns
        #replaced 'p_lessthan_alpha' with 'sig'
        new_col_names = [col.replace('p',"sig") for col in p_cols]
        try:
            #if there are multiple new columns (p_lessthan_alpha is a DataFrame)
            p_lessthan_alpha.columns = new_col_names#[i+" p_lessthan_alpha" for i in new_cols]
        except:
            #if there was only one new column (p_lessthan_alpha is a Series)
            p_lessthan_alpha.name = new_col_names#"p_lessthan_alpha"#.format(new_alpha)
        
        # concat new column(s) to old dataframe
        fully_analyzed = pd.concat([frame, p_lessthan_alpha],axis=1)
        compared_frames.append(fully_analyzed)
    return compared_frames


### make statistical significance frames

####################### end function defns ###########################

prism_out_dir = "/Users/morganfine-morris/Documents/Repos/NMProject/Paper/Data and Analysis/short TB Yan/Prism Output"
files = [i for i in os.listdir(prism_out_dir) if i.endswith('.csv') and not ('inv' in i) and not ('t_vals' in i)][1:]
print files
output_path = "/Users/morganfine-morris/Documents/Repos/NMProject/Paper/Data and Analysis/"

['2way ANOVA of Interburst Interval.csv', '2way ANOVA of Intraburst Freq.csv', '2way ANOVA of Peaks Amplitude.csv', '2way ANOVA of Peaks Interval.csv', '2way ANOVA of Peaks per Burst.csv', '2way ANOVA of Total Cycle Time.csv']


## For all files, save out t values

In [87]:
def save_t_vals_to_csv(prism_out_dir, filename):
    path = os.path.join(prism_out_dir, filename)
    table = get_table(path)
    key_list, t_vals_dict = collect_t_vals(table) #key_list is to retain original order
    data = [t_vals_dict[key_str] for key_str in key_list]
    t_vals = pd.DataFrame(data, index=key_list, columns=['TB: t','Yan: t'])
    
    #save t-vals
    t_val_file = os.path.join(prism_out_dir, "t_vals from "+filename)
    t_vals.to_csv(t_val_file)
    return t_val_file
    
for i in files:
    print save_t_vals_to_csv(prism_out_dir, i)
    pass

/Users/morganfine-morris/Documents/Repos/NMProject/Paper/Data and Analysis/short TB Yan/Prism Output/t_vals from 2way ANOVA of Interburst Interval.csv
/Users/morganfine-morris/Documents/Repos/NMProject/Paper/Data and Analysis/short TB Yan/Prism Output/t_vals from 2way ANOVA of Intraburst Freq.csv
/Users/morganfine-morris/Documents/Repos/NMProject/Paper/Data and Analysis/short TB Yan/Prism Output/t_vals from 2way ANOVA of Peaks Amplitude.csv
/Users/morganfine-morris/Documents/Repos/NMProject/Paper/Data and Analysis/short TB Yan/Prism Output/t_vals from 2way ANOVA of Peaks Interval.csv
/Users/morganfine-morris/Documents/Repos/NMProject/Paper/Data and Analysis/short TB Yan/Prism Output/t_vals from 2way ANOVA of Peaks per Burst.csv
/Users/morganfine-morris/Documents/Repos/NMProject/Paper/Data and Analysis/short TB Yan/Prism Output/t_vals from 2way ANOVA of Total Cycle Time.csv


## For one Measure

In [204]:
measures = ['Burst Duration',
'Interburst Interval',
'Intraburst Freq',
'Peaks Amplitude',
'Peaks Interval',
'Peaks per Burst',
'Total Cycle Time']
measure_type = measures[-2]
#print "Measurement Type: ",measure_type

for measure_type in measures:
    df_param = real_ps_1(measure_type)
    df_model = real_ps_2(measure_type)
    #compared_dfs = compare_p_against_newalpha([df,df_2],['*','ns'])#this will replace True with * and False with ns
    compared_dfs = compare_p_against_newalpha([df_param,df_model])#this will give pure true-false
    df_param = compared_dfs[0]
    df_model = compared_dfs[1] 
    print df_param.columns
    df_model.to_csv(path.join(prism_out_dir,'model_comparison_{}.csv').format(measure_type))
    df_param.to_csv(path.join(prism_out_dir,'param_comparison_{}.csv').format(measure_type))

Index([u'TB', u'Yan', u'TB p', u'Yan p', u'TB sig', u'Yan sig'], dtype='object')
Index([u'TB', u'Yan', u'TB p', u'Yan p', u'TB sig', u'Yan sig'], dtype='object')
Index([u'TB', u'Yan', u'TB p', u'Yan p', u'TB sig', u'Yan sig'], dtype='object')
Index([u'TB', u'Yan', u'TB p', u'Yan p', u'TB sig', u'Yan sig'], dtype='object')
Index([u'TB', u'Yan', u'TB p', u'Yan p', u'TB sig', u'Yan sig'], dtype='object')
Index([u'TB', u'Yan', u'TB p', u'Yan p', u'TB sig', u'Yan sig'], dtype='object')
Index([u'TB', u'Yan', u'TB p', u'Yan p', u'TB sig', u'Yan sig'], dtype='object')


In [12]:
#df1[df1['Yan p_lessthan_alpha']==False].index.values
#df1[df1['TB p_lessthan_alpha']==False].index.values

In [196]:
df2.to_csv(path.join(prism_out_dir,'model_comparison_{}.csv').format(measure_type))
df1.to_csv(path.join(prism_out_dir,'param_comparison_{}.csv').format(measure_type))

In [194]:
os.getcwd()

'/Users/morganfine-morris/Documents/Repos/NMProject/IPYNB'

## Make param comparison table

In [27]:
def make_param_compare_table_multiind(measurement, split_on=' vs '):
    '''
    measurement is a series with data of just one measurement type (burst duration) for a single model
    Its rows should be in format:
    <param_1_val><param_1_name> <param_2_val><param_2_name>
    or 
    <param_1_name><param_1_val> <param_2_name><param_2_val>
    '''
    
    import re
    r1 = re.compile("([\-][\d+.\d+]+|[\d+.\d+]+)([a-zA-Z]+)")
    r2 = re.compile("([a-zA-Z]+)([\-][\d+.\d+]+|[\d+.\d+]+)")
    
    temp ={}
    for k,v in measurement.iteritems():
        #v is row value, k is row label
        
        p1KEY, p2KEY =  k.split(split_on)
        e1 = p1KEY[2:5]
        e2 = p2KEY[2:5]
        g1 = p1KEY[13:]
        g2 = p2KEY[13:]
        #print e1, e2
        #print g1, g2
        #print
#        if not v:
        '''
        if (p1KEY[2:5] == p2KEY[2:5]):
            print p1KEY[2:5],") ", p1KEY[13:], 'vs', p2KEY[13:]
        elif (p1KEY[13:] == p2KEY[13:]):
            print p1KEY[13:],") ", p1KEY[2:5] ,"vs", p2KEY[2:5]
        else:
            print p1KEY, ' vs ', p2KEY
            
            #print p1KEY, ' vs ', p2KEY
        '''
make_param_compare_table_multiind(df1['TB p_lessthan_alpha'])


In [30]:
def make_param_compare_table(measurement, split_on=' vs '):
    '''
    measurement is a series with data of just one measurement type (burst duration) for a single model
    Its rows should be in format:
    <param_1_val><param_1_name> <param_2_val><param_2_name>
    or 
    <param_1_name><param_1_val> <param_2_name><param_2_val>
    '''
    
    import re
    r1 = re.compile("([\-][\d+.\d+]+|[\d+.\d+]+)([a-zA-Z]+)")
    r2 = re.compile("([a-zA-Z]+)([\-][\d+.\d+]+|[\d+.\d+]+)")
    
    temp ={}
    for k,v in measurement.iteritems():
        #v is row value, k is row label
        
        p1KEY, p2KEY =  k.split(split_on)
        e1 = p1KEY[2:5]
        e2 = p2KEY[2:5]
        g1 = p1KEY[13:]
        g2 = p2KEY[13:]
        #p1KEY = e1+" "+g1
        #p2KEY = e2+" "+g2

        """
        if not v:
            if (p1KEY[2:5] == p2KEY[2:5]):
                print p1KEY[2:5],") ", p1KEY[13:], 'vs', p2KEY[13:]
            elif (p1KEY[13:] == p2KEY[13:]):
                print p1KEY[13:],") ", p1KEY[2:5] ,"vs", p2KEY[2:5]
            else:
                print p1KEY, ' vs ', p2KEY
            
            #print p1KEY, ' vs ', p2KEY

 
        
        
        if not (p1KEY[2:5] == p2KEY[2:5]) or (p1KEY[13:] == p2KEY[13:]):
            print p1KEY, ' vs ', p2KEY,
            if v:
                print ": ", v
            else:
                print
            pass
        else:
            continue
        """
        
        
        try:
            #append to current sub-dict
            temp[p1KEY][p2KEY] = v

            #temp[p2KEY][p1KEY] = v
            #print p1KEY, p2KEY
            #temp[p2KEY][p1KEY] = v
        except:
            #s = pd.Series(temp[p1KEY]).sort(ascending=False, inplace=False)
            #print s
            #start new sub-dict
            temp[p1KEY] = {}
            temp[p1KEY][p1KEY] = np.nan
            temp[p1KEY][p2KEY] = v



            #print pd.DataFrame.from_dict(temp)
            #print 'keys:',p1KEY, p2KEY
            #print

    return pd.DataFrame.from_dict(temp)
print measure_type
print "TB:"
tb = make_param_compare_table(df1['TB p_lessthan_alpha'])
print
print "Yan:"
yan = make_param_compare_table(df1['Yan p_lessthan_alpha'])
combined = yan.T.combine_first(tb).replace(to_replace=[False, True, np.nan], value=['ns', '*', ' '])
str_combined = combined.to_csv()
#combined

Peaks per Burst
TB:

Yan:
,eL-50.0 gnaps1.8,eL-50.0 gnaps2.4,eL-55.0 gnaps1.8,eL-55.0 gnaps2.4,eL-55.0 gnaps3.0,eL-55.0 gnaps3.6,eL-55.0 gnaps4.2,eL-60.0 gnaps1.8,eL-60.0 gnaps2.4,eL-60.0 gnaps3.0,eL-60.0 gnaps3.6,eL-60.0 gnaps4.2,eL-65.0 gnaps1.8,eL-65.0 gnaps2.4,eL-65.0 gnaps3.0,eL-65.0 gnaps3.6,eL-65.0 gnaps4.2
eL-50.0 gnaps1.8, ,ns,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
eL-50.0 gnaps2.4,*, ,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
eL-55.0 gnaps1.8,*,*, ,*,*,*,*,*,ns,*,*,*,*,*,*,*,*
eL-55.0 gnaps2.4,*,*,*, ,*,*,*,*,*,ns,*,*,*,*,*,*,*
eL-55.0 gnaps3.0,*,*,*,*, ,*,*,*,*,*,*,ns,*,*,*,*,*
eL-55.0 gnaps3.6,*,*,*,*,*, ,*,*,*,*,*,*,*,*,*,*,*
eL-55.0 gnaps4.2,*,*,*,*,*,*, ,*,*,*,*,*,*,*,*,*,*
eL-60.0 gnaps1.8,*,*,*,*,*,*,*, ,*,*,*,*,*,*,*,*,*
eL-60.0 gnaps2.4,*,*,*,*,*,*,*,*, ,*,*,*,*,*,*,*,*
eL-60.0 gnaps3.0,*,*,*,*,*,*,*,*,*, ,*,*,*,*,*,*,*
eL-60.0 gnaps3.6,*,*,*,*,*,*,*,*,*,*, ,*,*,*,*,ns,*
eL-60.0 gnaps4.2,*,*,ns,*,*,*,*,*,*,*,*, ,*,*,*,*,*
eL-65.0 gnaps1.8,*,*,*,*,*,*,*,*,*,*,*,*, ,*,*,*,*
eL-65.0 gnaps2.

In [16]:
#save param compare table
lines = str_combined.split('\n')
lines[1] = lines[1]+',TB'
ncommas = lines[0].count(',')
line =',Yan'+"".join([','for i in range(ncommas)])
lines[-1]=line
filepath = path.join(output_path,'Parameter Compare Summary_{}.csv'.format(measure_type))
with open(filepath,'w') as f:
    f.write('\n'.join(lines))

In [ ]:
heatmappath = os.path.join(prism_out_dir,'plots','heatmap {}.png'.format(key))



## Make Table to compare models

In [310]:
#load data if necessary
model_Comp = pd.DataFrame.from_csv('model_comparison_{}.csv'.format(measure_type))
model_Comp

,TB vs Yan t,TB vs Yan real p,TB vs Yan p_lessthan_alpha
eL-65.0 gnaps4.2,23.2400,9.342810e-14,True
eL-50.0 gnaps2.4,32.9200,3.968334e-16,True
eL-65.0 gnaps3.0,1.9030,7.519211e-02,False
eL-60.0 gnaps1.8,0.1169,9.083941e-01,False
eL-55.0 gnaps1.8,27.9900,5.101168e-15,True
eL-60.0 gnaps3.0,28.3400,4.196924e-15,True
eL-60.0 gnaps4.2,115.9000,7.885285e-25,True
eL-55.0 gnaps4.2,21.5700,2.975094e-13,True
eL-60.0 gnaps2.4,11.6700,3.078367e-09,True
eL-60.0 gnaps3.6,28.2600,4.387435e-15,True


In [311]:
#load model comparison file and extract the comprison column. replace NaN with False
try:
    booleans = model_Comp['TB vs Yan p_lessthan_alpha']
except:
    booleans = model_Comp['Yan vs TB p_lessthan_alpha']


stat_sig_frame = NMUtility.make_frame(booleans).fillna('')
stat_sig_frame

#replace True with '*' and False with 'ns'
starred_frame = stat_sig_frame.replace(to_replace=[False, True], value=['ns','*'])
starred_frame

,-65.0,-60.0,-55.0,-50.0
1.8,ns,ns,*,*
2.4,ns,*,*,*
3.0,ns,*,*,
3.6,*,*,*,
4.2,*,*,*,


In [312]:
starred_frame.to_csv("Model Compare Summary {}.csv".format(measure_type))

In [17]:
NMUtility.make_frame??

# STOP!!!!

In [93]:
n=0
for i in s:
    i1,i2 = [ii.strip() for ii in i.split('vs')]
    #print "'{}'".format(i1),"'{}'".format(i2)
    #print i1.split(' ')[0:2],":" ,i2.split(' ')[0:2]
    el1, gn1 = i1.split(' ')[0:2]
    el2, gn2 = i2.split(' ')[0:2]
    #print ">", el1, el2
    #print ">", gn1, gn2
    if (gn1 == gn2) or (el1 == el2):
        n += 1
        if (el1 == el2):
            #print i1, ", ",i2
            print el1, "    ", gn1,"==", gn2
        if (gn1 == gn2):
            print gn1, "    ", el1, "==", el2
print n

eL=-55.0      gnaps=1.8 == gnaps=2.4
eL=-55.0      gnaps=1.8 == gnaps=3.0
eL=-55.0      gnaps=1.8 == gnaps=3.6
eL=-55.0      gnaps=1.8 == gnaps=4.2
gnaps=1.8      eL=-55.0 == eL=-60.0
gnaps=1.8      eL=-55.0 == eL=-65.0
eL=-55.0      gnaps=2.4 == gnaps=3.0
eL=-55.0      gnaps=2.4 == gnaps=3.6
eL=-55.0      gnaps=2.4 == gnaps=4.2
gnaps=2.4      eL=-55.0 == eL=-60.0
gnaps=2.4      eL=-55.0 == eL=-65.0
eL=-55.0      gnaps=3.0 == gnaps=3.6
eL=-55.0      gnaps=3.0 == gnaps=4.2
gnaps=3.0      eL=-55.0 == eL=-60.0
gnaps=3.0      eL=-55.0 == eL=-65.0
eL=-55.0      gnaps=3.6 == gnaps=4.2
gnaps=3.6      eL=-55.0 == eL=-60.0
gnaps=3.6      eL=-55.0 == eL=-65.0
gnaps=4.2      eL=-55.0 == eL=-60.0
gnaps=4.2      eL=-55.0 == eL=-65.0
eL=-60.0      gnaps=1.8 == gnaps=2.4
eL=-60.0      gnaps=1.8 == gnaps=3.0
eL=-60.0      gnaps=1.8 == gnaps=3.6
eL=-60.0      gnaps=1.8 == gnaps=4.2
gnaps=1.8      eL=-60.0 == eL=-65.0
eL=-60.0      gnaps=2.4 == gnaps=3.0
eL=-60.0      gnaps=2.4 == gnaps=3.6
eL=-60.0    